# **Updated population density of the LA city neighborhood councils.**
    
### Here is a notebook to evaluate the populataion, area(in square miles) and population density of the 99 Neighborhood councils (NCs). 

In [1]:
# Importing the necessary packages.
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import folium
from shapely.geometry import Point, Polygon 
from pyproj import Geod
import numpy as np
import webbrowser

In [2]:
# Path to the directory.
dir = 'C:/Users/AdithiPriya/Desktop/Hack for LA/Geospatial analysis'

# Accessing the Census 2020 data.
census_2020 = gpd.read_file(os.path.join(dir, 'tl_2020_06037_tract20/tl_2020_06037_tract20.shp'))

# Reading the LA NC shape file.
la_nc = gpd.read_file(os.path.join(dir, 'Neighborhood_Councils_(Certified)/Neighborhood_Councils_(Certified).shp'))

# ACS demographics data.
acs_demo = pd.read_csv(os.path.join(dir,'ACS_census_tract_LA.csv' ))
acs_demo['GEO_ID']= acs_demo['GEO_ID'].str.replace('1400000US','')

# Rename GEO_ID as GEOID20.

acs_demo= acs_demo.rename(columns={'GEO_ID':'GEOID20', 'Total population': 'population'})

### Defining a [function](https://github.com/hackforla/access-the-data-workshop-311-analysis/blob/main/notebooks/NC-population-density.ipynb) to compute the area in square miles (area (sq_miles)).

In [3]:
geod = Geod(ellps= 'WGS84') 

def area_sq_miles(geo):
    area_sq_meters= abs(geod.geometry_area_perimeter(geo)[0])
    return (area_sq_meters * 3.86102e-7)

In [4]:
# Taking a subset of the acs_demo dataframe.

acs_demo_subset=acs_demo[['GEOID20','population' ]]

# Making sure that the crs- coordinate reference system for census_2020 is the same as that of the la_nc using the to_crs()
# method. This will allow the spatial merging of both the geopandas dataframes.

census_2020 = census_2020.to_crs(la_nc.crs)

# Spatial overlap of the la_nc and census_2020 data.

census_NC = gpd.overlay(census_2020, la_nc, how='intersection')

census_NC= census_NC[['TRACTCE20','GEOID20', 'NAME', 'NC_ID', 'INTPTLAT20', 'INTPTLON20', 'geometry']]
print('Number of rows in census_NC: ', census_NC.shape[0])  

Number of rows in census_NC:  2108


In [5]:
# Let us take a look at the spatially merged data:
census_NC.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2..."
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2..."
2,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2..."
3,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2..."
4,101222,06037101222,SUNLAND-TUJUNGA NC,10,+34.2513519,-118.2885261,"POLYGON ((-118.29434 34.25233, -118.29318 34.2..."


In [6]:
# Merging the census_NC and acs_demo_subset.
census = pd.merge(census_NC, acs_demo_subset, on='GEOID20')
census.shape[0]

2108

### Displaying the duplicate entries- census tracts intersecting more than 1 Neighborhood councils

In [7]:
df_duplicate = census_NC[census_NC.GEOID20.duplicated(keep=False)]
df_duplicate.sort_values(by=['GEOID20']).head(15)

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2..."
17,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2..."
5,101300,06037101300,SUNLAND-TUJUNGA NC,10,+34.2487777,-118.2709990,"POLYGON ((-118.27822 34.25068, -118.27822 34.2..."
18,101300,06037101300,FOOTHILL TRAILS DISTRICT NC,9,+34.2487777,-118.2709990,"POLYGON ((-118.26682 34.23124, -118.26695 34.2..."
6,101400,06037101400,SUNLAND-TUJUNGA NC,10,+34.2428521,-118.2941612,"POLYGON ((-118.32227 34.24961, -118.32212 34.2..."
19,101400,06037101400,FOOTHILL TRAILS DISTRICT NC,9,+34.2428521,-118.2941612,"POLYGON ((-118.32238 34.24963, -118.32227 34.2..."
42,102103,06037102103,SUN VALLEY AREA NC,8,+34.2250792,-118.3541880,"POLYGON ((-118.36533 34.22870, -118.36396 34.2..."
20,102103,06037102103,FOOTHILL TRAILS DISTRICT NC,9,+34.2250792,-118.3541880,"POLYGON ((-118.35739 34.22856, -118.35546 34.2..."
43,102104,06037102104,SUN VALLEY AREA NC,8,+34.2161873,-118.3453981,"POLYGON ((-118.35620 34.21971, -118.35594 34.2..."
21,102104,06037102104,FOOTHILL TRAILS DISTRICT NC,9,+34.2161873,-118.3453981,"MULTIPOLYGON (((-118.34413 34.21387, -118.3441..."


In [8]:
df_duplicate.shape[0]

1475

### Out of 2108 rows, 1475 rows are duplicate. 

### Here is a map of NCs and census tracts. Zoom in and toggle between NC and census tracts to see that some of the NCs share the census tracts with their neighbors. 

### Here is a link to the [map](https://exquisite-squirrel-d3afcc.netlify.app/). I am using netlify to deploy the interactive map. 

In [9]:
m= la_nc.explore(
    column= 'NAME', # make choropleth based on 'NC name' column
    name='NC Regions', 
    tooltip='NAME', # show 'NC name' value in tooltip (on hover)
    color="black", # use red color on all points
    popup=True, # show all values in popup (on click)
    tiles="openstreetmap", # use "openstreetmap" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"), # use black outline
    legend=False
     )
census_NC.explore(
    m=m,
    column= 'TRACTCE20', # make choropleth based on 'NC name' column
    name='Census tracts', 
    tooltip='TRACTCE20', # show 'NC name' value in tooltip (on hover)
    color="red", # use red color on all points
    popup=True, # show all values in popup (on click)
    tiles="openstreetmap", # use "openstreetmap" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"), # use black outline
    legend=False
     )
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m.save('census_plus_NC.html')
webbrowser.open('census_plus_NC.html')

True

### Before moving forward, for sanity check, let us try to get the population of distinct census tracts without worrying about the proportion of NC areas intersected by census tracts. To do this, I will retain the first row of the  duplicate data for the entire dataframe.

In [10]:
test =  census.groupby('TRACTCE20').first()
test.population.sum()

4534712

<div style="text-align: justify">  
    
### This number exactly matches with the one that I get using this [notebook](https://github.com/priyakalyan/Updated_NC_pop/blob/main/NC_pop_recent_no_filter.ipynb) where the population of NCs have been calculated at the tract level. This implies that the census tracts that is being used in the calculation is wrong. This issue will be addressed here. But before going ahead, the next section will outline the methodology to calculate the area of census tracts intersecting multiple NCs.

### So here is a method to take care of cases where the census tract intersects more than 1 NCs. 

- **Find the area for each entry in census dataframe.** 
- **Group the census dataframe by 'TRACTCE20' and then find sum of the area- this gives you the total area of each census       tract.** 
- **Next, find the percentage of area of the census tract intersecting different NCs.** 
- **Use this information to find the percentage of population for each census tract.**
- **Then find the total population by grouping them by NCs.** 

In [11]:
# Add the area column.
census = pd.merge(census_NC, acs_demo_subset, on='GEOID20')
census['area(sq_miles)']= census.apply(lambda x: area_sq_miles(x.geometry), axis=1)
census[['NC_ID','NAME', 'TRACTCE20', 'population','area(sq_miles)']].head(10)

,NC_ID,NAME,TRACTCE20,population,area(sq_miles)
0,10,SUNLAND-TUJUNGA NC,101110,3923,0.441083
1,10,SUNLAND-TUJUNGA NC,101122,4119,1.020872
2,9,FOOTHILL TRAILS DISTRICT NC,101122,4119,0.000038
3,10,SUNLAND-TUJUNGA NC,101220,3775,0.269841
4,10,SUNLAND-TUJUNGA NC,101221,3787,0.136748
5,10,SUNLAND-TUJUNGA NC,101222,2717,0.114484
6,10,SUNLAND-TUJUNGA NC,101300,3741,0.993003
7,9,FOOTHILL TRAILS DISTRICT NC,101300,3741,0.002387
8,10,SUNLAND-TUJUNGA NC,101400,3246,2.414663
9,9,FOOTHILL TRAILS DISTRICT NC,101400,3246,0.021664


In [12]:
census_test=census.groupby('TRACTCE20', as_index= False).agg({'NC_ID':'first','NAME':'first','geometry':'first', 'INTPTLAT20':'first', 'INTPTLON20':'first', 'NAME':'first','GEOID20':'first','area(sq_miles)':sum})
census_test.rename(columns={'area(sq_miles)': 'total_area(sq_miles)'}, inplace=True)
census_test= census_test[['TRACTCE20','total_area(sq_miles)']]

In [13]:
# Let us add the total_area to census dataframe.
census_perc= pd.merge(census, census_test, on='TRACTCE20')
census_perc.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry,population,area(sq_miles),total_area(sq_miles)
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",3923,0.441083,0.441083
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4119,1.020872,1.020910
2,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2...",4119,0.000038,1.020910
3,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",3775,0.269841,0.269841
4,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",3787,0.136748,0.136748


In [14]:
# Evaluating the percentage of the intersecting areas (area_perc):
census_perc['area_perc'] = census_perc['area(sq_miles)']/census_perc['total_area(sq_miles)']

# Adding the percentage of population column.
census_perc['total_population'] = census_perc['population']*census_perc['area_perc'] 

# Rounding the population number:
census_perc['total_population'] = census_perc['total_population'].apply(np.floor)

# Converting the population to integer type.

census_perc['total_population']  = census_perc['total_population'].astype(int)

census_perc[['TRACTCE20', 'NAME', 'NC_ID', 'population', 'area(sq_miles)', 'area_perc', 'total_area(sq_miles)', 'total_population']].head(15)

,TRACTCE20,NAME,NC_ID,population,area(sq_miles),area_perc,total_area(sq_miles),total_population
0,101110,SUNLAND-TUJUNGA NC,10,3923,0.441083,1.000000,0.441083,3923
1,101122,SUNLAND-TUJUNGA NC,10,4119,1.020872,0.999963,1.020910,4118
2,101122,FOOTHILL TRAILS DISTRICT NC,9,4119,0.000038,0.000037,1.020910,0
3,101220,SUNLAND-TUJUNGA NC,10,3775,0.269841,1.000000,0.269841,3775
4,101221,SUNLAND-TUJUNGA NC,10,3787,0.136748,1.000000,0.136748,3787
5,101222,SUNLAND-TUJUNGA NC,10,2717,0.114484,1.000000,0.114484,2717
6,101300,SUNLAND-TUJUNGA NC,10,3741,0.993003,0.997602,0.995390,3732
7,101300,FOOTHILL TRAILS DISTRICT NC,9,3741,0.002387,0.002398,0.995390,8
8,101400,SUNLAND-TUJUNGA NC,10,3246,2.414663,0.991108,2.436327,3217
9,101400,FOOTHILL TRAILS DISTRICT NC,9,3246,0.021664,0.008892,2.436327,28


<div style="text-align: justify">  
    
### I will be incorporating the area filter and the population filter to skim out the values that were causing the population of the NCs to inflate. 

### Let us take a look at the NC- Atwater Village. 

In [15]:
census_perc[census_perc['NAME'] == 'ATWATER VILLAGE NC'][['TRACTCE20', 'NAME', 'NC_ID', 'population', 'area(sq_miles)', 'area_perc', 'total_area(sq_miles)', 'total_population']]

,TRACTCE20,NAME,NC_ID,population,area(sq_miles),area_perc,total_area(sq_miles),total_population
811,189703,ATWATER VILLAGE NC,37,1880,0.043906,0.029314,1.497769,55
833,311601,ATWATER VILLAGE NC,37,2608,0.000209,0.149092,0.001404,388
837,311700,ATWATER VILLAGE NC,37,5966,0.004878,0.322874,0.015109,1926
841,980009,ATWATER VILLAGE NC,37,5,6.816197,0.312739,21.795157,1
973,187102,ATWATER VILLAGE NC,37,4376,0.340444,0.402493,0.845840,1761
984,187101,ATWATER VILLAGE NC,37,3213,0.302714,0.995004,0.304234,3196
986,187200,ATWATER VILLAGE NC,37,3324,0.011649,0.031713,0.367315,105
990,187300,ATWATER VILLAGE NC,37,3715,0.004740,0.010301,0.460138,38
994,188100,ATWATER VILLAGE NC,37,3960,0.813095,1.000000,0.813095,3960
995,188201,ATWATER VILLAGE NC,37,3262,0.019628,0.058066,0.338025,189


### Look at the census tracts 301601, 301602, 301702, 311801. When I looked at the map, these were the tracts belonging to the neighboring city- Glendale. These tracts obviously do not belong to Atwater Village NC. The total area of these tracts are extremely small, but the area_perc for some of these tracts - 1 and so the whole population is taken into account and thus this will result in a higher population value than the expected one. So, I am adding the area filter- total_area(sq_miles) >= 0.01. 
    
### Similarly, some of the population value is so small, but the cumulative effect can result in an exaggerated population number. Therefore, I am going to add the population filter here. The [minimum population](https://www2.census.gov/geo/pdfs/education/CensusTracts.pdf) of a whole census tract is 1200. Since we are dealing with proportional areas and sometimes, a census tract intersects 2 NCs in 30% to 70% ratio and so I want to be reasonable with the population filter- 1000 (we do not want to undercount too). 

In [16]:
# Filtering out total_area < 0.01 square miles.
census_perc_filter = census_perc[census_perc['total_area(sq_miles)'] >= 0.01]
census_perc_filter.head()

,TRACTCE20,GEOID20,NAME,NC_ID,INTPTLAT20,INTPTLON20,geometry,population,area(sq_miles),total_area(sq_miles),area_perc,total_population
0,101110,06037101110,SUNLAND-TUJUNGA NC,10,+34.2594737,-118.2929869,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",3923,0.441083,0.441083,1.000000,3923
1,101122,06037101122,SUNLAND-TUJUNGA NC,10,+34.2677213,-118.2901465,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4119,1.020872,1.020910,0.999963,4118
2,101122,06037101122,FOOTHILL TRAILS DISTRICT NC,9,+34.2677213,-118.2901465,"POLYGON ((-118.29785 34.27778, -118.29783 34.2...",4119,0.000038,1.020910,0.000037,0
3,101220,06037101220,SUNLAND-TUJUNGA NC,10,+34.2516083,-118.2816328,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",3775,0.269841,0.269841,1.000000,3775
4,101221,06037101221,SUNLAND-TUJUNGA NC,10,+34.2543290,-118.2925767,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",3787,0.136748,0.136748,1.000000,3787


In [17]:
# Removing the rows with population value < 500.
census_filter = census_perc_filter[census_perc_filter['total_population'] >= 1000]
census_filter.shape[0]

1172

In [18]:
# Summming up the population of each NCs.
census_pop =census_filter.groupby('NAME', as_index= False).agg({'NC_ID':'first',  'NAME':'first', 'total_population' : sum})
census_pop['total_population'] = census_pop['total_population'].astype(int)
census_pop.head(50)

,NC_ID,NAME,total_population
0,6,ARLETA NC,35585
1,42,ARROYO SECO NC,19479
2,46,ARTS DISTRICT LITTLE TOKYO NC,4153
3,37,ATWATER VILLAGE NC,14665
4,64,BEL AIR-BEVERLY CREST NC,24732
5,50,BOYLE HEIGHTS NC,87817
6,13,CANOGA PARK NC,54184
7,110,CENTRAL ALAMEDA NC,30219
8,32,CENTRAL HOLLYWOOD NC,18204
9,95,CENTRAL SAN PEDRO NC,29244


In [19]:
census_pop.total_population.sum()

3825667

<div style="text-align: justify">  
    
### This number is close to the [Census Bureau](https://www.census.gov/quickfacts/losangelescitycalifornia?) value- 3,849,297. Now that we have the population of each NC, let us move on to getting the area and finally the updated population density of the neighborhood councils.      

In [20]:
# Grouping the original census dataframe by NAME and then summing up the total area- this gives the area of each neighborhood 
# council. Very important note here- groupby function works when trying to aggregate dataframes but for spatial data, we can
# aggregate the geometry features using dissolve function.

census_area = census.dissolve(by= 'NAME', as_index= False, aggfunc=({'NC_ID':'first','area(sq_miles)' : sum }))

# Gathering all the columns of interest. 
census_final = census_area.join(census_pop['total_population'])
census_final['pop_density']= census_final['total_population']/census_final['area(sq_miles)']

# Rearranging the columns.
census_final = census_final[['NAME', 'geometry', 'NC_ID', 'total_population', 'area(sq_miles)', 'pop_density']]
census_final.head(15)

,NAME,geometry,NC_ID,total_population,area(sq_miles),pop_density
0,ARLETA NC,"POLYGON ((-118.41010 34.23309, -118.41034 34.2...",6,35585,3.284868,10833.007723
1,ARROYO SECO NC,"POLYGON ((-118.18576 34.09293, -118.18576 34.0...",42,19479,3.063327,6358.773341
2,ARTS DISTRICT LITTLE TOKYO NC,"POLYGON ((-118.22877 34.04155, -118.22827 34.0...",46,4153,0.879216,4723.528997
3,ATWATER VILLAGE NC,"POLYGON ((-118.25399 34.10816, -118.25424 34.1...",37,14665,8.748450,1676.296980
4,BEL AIR-BEVERLY CREST NC,"POLYGON ((-118.46573 34.07325, -118.46581 34.0...",64,24732,17.038756,1451.514436
5,BOYLE HEIGHTS NC,"POLYGON ((-118.20504 34.01263, -118.20504 34.0...",50,87817,5.735881,15310.116685
6,CANOGA PARK NC,"POLYGON ((-118.58846 34.19524, -118.58846 34.1...",13,54184,3.689892,14684.440778
7,CENTRAL ALAMEDA NC,"POLYGON ((-118.23777 33.98933, -118.23777 33.9...",110,30219,1.358014,22252.346382
8,CENTRAL HOLLYWOOD NC,"POLYGON ((-118.32445 34.08712, -118.32445 34.0...",32,18204,1.229127,14810.517707
9,CENTRAL SAN PEDRO NC,"POLYGON ((-118.28794 33.73151, -118.28795 33.7...",95,29244,2.438025,11994.957379


### Here is a plot of census_final dataframe with the population, population density and area popping up for each NCs. I am posting the link to this [map](https://ephemeral-salamander-ff5d33.netlify.app/).

In [21]:
m1 = census_final.explore(
    column= 'NAME', # make choropleth based on 'NC name' column
    name='NC Regions', 
    tooltip='NAME', # show 'NC name' value in tooltip (on hover)
    color="black", # use red color on all points
    popup=True, # show all values in popup (on click)
    tiles="openstreetmap", # use "openstreetmap" tiles
    cmap="Set1", # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"), # use black outline
    legend=False
     )
folium.TileLayer('Stamen Terrain').add_to(m1)
folium.TileLayer('Stamen Toner').add_to(m1)
folium.TileLayer('Stamen Water Color').add_to(m1)
folium.TileLayer('cartodbpositron').add_to(m1)
folium.TileLayer('cartodbdark_matter').add_to(m1)
folium.LayerControl().add_to(m1)
m1.save('census_final.html')
webbrowser.open('census_final.html')

True

In [22]:
census_final.head()

,NAME,geometry,NC_ID,total_population,area(sq_miles),pop_density
0,ARLETA NC,"POLYGON ((-118.41010 34.23309, -118.41034 34.2...",6,35585,3.284868,10833.007723
1,ARROYO SECO NC,"POLYGON ((-118.18576 34.09293, -118.18576 34.0...",42,19479,3.063327,6358.773341
2,ARTS DISTRICT LITTLE TOKYO NC,"POLYGON ((-118.22877 34.04155, -118.22827 34.0...",46,4153,0.879216,4723.528997
3,ATWATER VILLAGE NC,"POLYGON ((-118.25399 34.10816, -118.25424 34.1...",37,14665,8.748450,1676.296980
4,BEL AIR-BEVERLY CREST NC,"POLYGON ((-118.46573 34.07325, -118.46581 34.0...",64,24732,17.038756,1451.514436


In [23]:
# Save the dataframe as a csv file after dropping the geometry column.
census_final= census_final.drop(['geometry'], axis=1)
census_final.to_csv('census_final.csv')